In [ ]:
import redis
import json
import time
import uuid
import inspect
import re

class RemoteSender:
    def __init__(self, host='220.203.1.124', port=6379, db=0,
                 task_queue='code_tasks', result_queue='execution_results',
                 password='***'):
        self.redis = redis.Redis(
            host=host, port=port, db=db, password=password,
            decode_responses=False
        )
        self.task_queue = task_queue
        self.result_queue = result_queue
        self._test_connection()

    def _test_connection(self):
        try:
            self.redis.ping()
            print("✅ 发送端已连接到Redis")
        except Exception as e:
            print(f"❌ 发送端连接失败: {e}")
            raise

    def _remove_decorators(self, func_code: str) -> str:
        lines = func_code.splitlines()
        cleaned_lines = []
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith('@') and not stripped_line.startswith('#'):
                continue
            cleaned_lines.append(line)
        return '\n'.join(cleaned_lines).strip('\n')

    def _normalize_indent(self, func_code: str) -> str:
        lines = func_code.splitlines()
        if not lines:
            return func_code
        
        min_indent = None
        for line in lines:
            stripped = line.lstrip()
            if stripped:
                indent = len(line) - len(stripped)
                if min_indent is None or indent < min_indent:
                    min_indent = indent
        
        if min_indent is not None and min_indent > 0:
            normalized_lines = []
            for line in lines:
                if len(line) >= min_indent and line[:min_indent].isspace():
                    normalized_lines.append(line[min_indent:])
                else:
                    normalized_lines.append(line)
            return '\n'.join(normalized_lines)
        else:
            return func_code

    def remote_task(self, func):
        def wrapper(*args, **kwargs):
            # 提取并清理代码
            func_code = inspect.getsource(func)
            cleaned_code = self._remove_decorators(func_code)
            final_code = self._normalize_indent(cleaned_code)
            
            # 生成任务ID并发送
            task_id = f"task_{uuid.uuid4().hex[:8]}"
            task = {
                'task_id': task_id,
                'func_code': final_code,
                'args': args,
                'kwargs': kwargs,
                'timestamp': time.time()
            }
            self.redis.rpush(self.task_queue, json.dumps(task, ensure_ascii=False))
            print(f"📤 已发送任务: {task_id}")
            
            # 获取结果
            return self._get_result(task_id)
        return wrapper

    def _get_result(self, task_id: str, timeout=60):
        start_time = time.time()
        while time.time() - start_time < timeout:
            result_data = self.redis.blpop(self.result_queue, timeout=5)
            if result_data:
                _, res_bytes = result_data
                result = json.loads(res_bytes.decode('utf-8'))
                if result['task_id'] == task_id:
                    if result['status'] == 'success':
                        return result['result']
                    else:
                        raise Exception(f"任务 {task_id} 执行失败: {result['error']}")
                else:
                    self.redis.rpush(self.result_queue, json.dumps(result, ensure_ascii=False))
            time.sleep(1)
        raise TimeoutError(f"任务 {task_id} 超时未返回")

# 使用示例
if __name__ == "__main__":
    sender = RemoteSender()

    # 1. 简单加法函数
    @sender.remote_task
    def simple_add(a, b):
        print(f"计算 {a} + {b}")
        return a + b

    # 2. 复杂计算函数
    @sender.remote_task
    def complex_function(a, b):
        print(f"计算 {a} 和 {b} 的复杂逻辑")
        if a > b:
            result = a **2 + b
            if result > 100:
                result /= 2
        else:
            result = (a + b) * 0.5
        print(f"结果: {result}")
        return result

    # 3. DataFrame处理函数
    @sender.remote_task
    def process_dataframe():
        import pandas as pd
        df = pd.DataFrame({
            'name': ['Alice', 'Bob'],
            'age': [25, 30]
        })
        print("创建的DataFrame:")
        print(df)
        return df

    # 执行测试
    try:
        print("=== 测试简单加法 ===")
        print("结果:", simple_add(10, 20))

        print("\n=== 测试复杂计算 ===")
        print("结果:", complex_function(15, 8))

        print("\n=== 测试DataFrame处理 ===")
        df_result = process_dataframe()
        print("接收的DataFrame数据:")
        print(f"类型: {df_result['type']}")
        print(f"列名: {df_result['columns']}")
        print(f"数据: {df_result['data']}")
    except Exception as e:
        print(f"❌ 执行失败: {e}")


✅ 发送端已连接到Redis
=== 测试简单加法 ===
📤 已发送任务: task_184e4773
结果: 30

=== 测试复杂计算 ===
📤 已发送任务: task_eff32708
结果: 116.5

=== 测试DataFrame处理 ===
📤 已发送任务: task_0af0e70c
接收的DataFrame数据:
类型: DataFrame
列名: ['index', 'name', 'age']
数据: [{'index': 0, 'name': 'Alice', 'age': 25}, {'index': 1, 'name': 'Bob', 'age': 30}]
